In [1]:
# -*- coding: utf-8 -*-
import os
import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup
from bs4 import element
from itertools import product
import numpy as np
import pandas as pd
from abbreviations import schwartz_hearst
from abbreviations import yanhu
import subprocess
from fuzzywuzzy import fuzz
print (schwartz_hearst.__file__)

/rds/general/user/yh4218/home/.conda/envs/NLP/lib/python3.8/site-packages/abbreviations/schwartz_hearst.py


/rds/general/user/yh4218/home/.conda/envs/NLP/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Get file path

In [2]:
base_dir = '/rdsgpfs/general/user/yh4218/home/data/NLP/NLP-MWAS'
#base_dir = '/rdsgpfs/general/user/yh4218/home/data/NLP/NLP-GWAS'

file_list = []
def get_files(base_dir,file_list):
    files = os.listdir(base_dir)
    for i in files:
        abs_path = os.path.join(base_dir,i)
        if re.match(r'(.*)PMC(.*).html',abs_path) and ('highlighted' not in abs_path):
            file_list.append(abs_path)
        elif os.path.isdir(abs_path)&('ipynb_checkpoints' not in abs_path):
            get_files(abs_path,file_list)
    return file_list

file_list = get_files(base_dir,file_list)

# Find/write html with a list of abbre

In [3]:
def find_write_html_with_abbre(soup_og):
    heading = soup_og.find_all('h2')
    abbre_exist=False
    for number, element in enumerate(heading):
        header = element.get_text()
        if re.search('abbreviation',header,re.IGNORECASE):
            abbre_exist=True
            directory=''
            for folder in file.split('/')[:-1]:
                directory +=folder+'/'
            path = directory.replace('data','outputs/abbre')
            try:os.makedirs(path)
            except:continue
            break
            
    if abbre_exist:
        with open(file.replace('data','outputs/abbre'),"w",encoding='UTF-8') as f:
            f.write(str(soup_og))

In [139]:
for file in file_list:
    with open(file,'r',encoding='UTF-8',errors='ignore') as f:
        text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
        find_write_html_with_abbre(soup)

# Read html with list of abbre

In [4]:
# MWAS
base_dir = '/rdsgpfs/general/user/yh4218/home/outputs/abbre/NLP/NLP-MWAS'
file_list=[]
file_list = get_files(base_dir,file_list)

#GWAS
#base_dir = '/rdsgpfs/general/user/yh4218/home/outputs/abbre/NLP/NLP-GWAS'
#file_list=[]
#file_list = get_files(base_dir,file_list)

### Extract abbre tables

In [5]:
def listToDict(lst):
    op = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return op
def abbre_table_to_dict(t):
    abbre_list=[]
    rows = t.findAll("tr")
    for i in rows:
        elements = i.findAll(['td', 'th'])
        vals = [j.get_text() for j in elements]
        abbre_list+=vals
    abbre_dict=listToDict(abbre_list)
    return abbre_dict
# old
def get_abbre_table_soup(soup_og):
    table = soup_og.find_all('table',recursive=True)
    table_soup=''
    for number, element in enumerate(table):
        nearest_tag = element.previous_element
        while nearest_tag:
            if nearest_tag.name != 'h2':
                nearest_tag = nearest_tag.previous_element
            else:
                if re.search('abbreviation',nearest_tag.get_text(),re.IGNORECASE):
                    table_soup=element
                break
    return table_soup
# Alex version
def get_abbre_table_soup_alex(soup_og):
    table = soup_og.find_all('table',recursive=True)
    table_soup=''
    for number, element in enumerate(table):
        nearest_tag = element.find_previous('h2','head')
        if re.search('abbreviation',nearest_tag.get_text(),re.IGNORECASE):
            table_soup=element
            break
    return table_soup

In [ ]:
output_f = open('/rdsgpfs/general/user/yh4218/home/outputs/abbre/list_of_abbre.txt','w',encoding='UTF-8')
for file in file_list:
    #print (file)
    output_f.write(file.split('/')[9]+'\n')
    with open(file,'r',encoding='UTF-8',errors='ignore') as f:
        text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
        table_soup=get_abbre_table_soup(soup)
        if table_soup:
            abbre_dict=abbre_table_to_dict(table_soup)
            #print (abbre_dict)
            #for SF, LF in abbre_dict:
            #    output_f.write(SF+'\t'+LF+'\n')
        else:
            #print ('failed')
            output_f.write('failed\n')
        f.close()
output_f.close()

### Extract abbre （table/list/plain text）

In [6]:
def listToDict(lst):
    op = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return op

def abbre_table_to_dict(t):
    abbre_list=[]
    rows = t.findAll("tr")
    for i in rows:
        elements = i.findAll(['td', 'th'])
        vals = [j.get_text() for j in elements]
        abbre_list+=vals
    abbre_dict=listToDict(abbre_list)
    return abbre_dict

def abbre_list_to_dict(t):
    abbre_list=[]
    SF = t.findAll("dt")
    SF_list = [SF_word.get_text() for SF_word in SF]
    LF = t.findAll("dd")
    LF_list = [LF_word.get_text() for LF_word in LF]
    abbre_dict=dict(zip(SF_list, LF_list))
    return abbre_dict

#########################################################################################
# below is back up
def get_abbre_plain_text(soup_og):
    abbre_text=soup_og.get_text()
    abbre_list=abbre_text.replace(':',';').replace(',',';').replace('.',';').split(';')
    return abbre_list
#########################################################################################
    
def get_abbre_plain_text(soup_og):
    abbre_text=soup_og.get_text()
    abbre_list=abbre_text.split(';')
    list_lenth=len(abbre_list)
    return abbre_list,list_lenth

####################### below is backup ###########################################
def get_abbre_dict(soup_og):
    header = soup_og.find_all('h2',recursive=True)
    abbre_dict={}
    for number, element in enumerate(header):
        if re.search('abbreviation',element.get_text(),re.IGNORECASE):
            nearest_down_tag = element.next_element
            while nearest_down_tag:
                tag_name = nearest_down_tag.name
                # when abbre is table
                if tag_name == 'table':
                    abbre_dict=abbre_table_to_dict(nearest_down_tag)
                    break
                    
                 # when abbre is list
                elif tag_name=='dl':
                    abbre_dict=abbre_list_to_dict(nearest_down_tag)
                    break
                
                 # when abbre is plain text
                elif tag_name=='p':
                    print ('——————————————————————'+PMC_number+'——————————————————————')
                    try:
                        abbre_list = get_abbre_plain_text(nearest_down_tag)
                        abbre_dict = listToDict(abbre_list)
                        for key,value in abbre_dict.items():
                            print (key+':'+value)
                            
                        break
                    except:
                        break
                    
                elif tag_name=='h2':
                    break
                    
                else:
                    nearest_down_tag = nearest_down_tag.next_element
                #print ('tag:'+str(nearest_down_tag)+'\nname:'+str(type(nearest_down_tag.name)))
                #nearest_down_tag = nearest_down_tag.next_element
    return abbre_dict
###################################################################################

def get_abbre_dict(soup_og,PMC_number):
    header = soup_og.find_all('h2',recursive=True)
    abbre_dict={}
    for number, element in enumerate(header):
        if re.search('abbreviation',element.get_text(),re.IGNORECASE):
            nearest_down_tag = element.next_element
            while nearest_down_tag:
                tag_name = nearest_down_tag.name
            # when abbre is table
                if tag_name == 'table':
                    abbre_dict=abbre_table_to_dict(nearest_down_tag)
                    break

            # when abbre is list
                elif tag_name=='dl':
                    abbre_dict=abbre_list_to_dict(nearest_down_tag)
                    break

            # when abbre is plain text
                elif tag_name=='p':
                    abbre_list,list_lenth = get_abbre_plain_text(nearest_down_tag)
                    if list_lenth<=2:
                        nearest_down_tag = nearest_down_tag.next_element
                        continue
                    else:
                        #print ('——————————————————————'+PMC_number+'——————————————————————')
                        for abbre_pair in abbre_list:
                            if len(abbre_pair.split(':'))==2:abbre_dict.update({abbre_pair.split(':')[0]:abbre_pair.split(':')[1]})
                            elif len(abbre_pair.split(','))==2:abbre_dict.update({abbre_pair.split(',')[0]:abbre_pair.split(',')[1]})
                            elif len(abbre_pair.split(' '))==2:abbre_dict.update({abbre_pair.split(' ')[0]:abbre_pair.split(' ')[1]})
                        #print (abbre_dict)
                        break

                        
            # search until next h2
                elif tag_name=='h2':
                    break
                else:
                    nearest_down_tag = nearest_down_tag.next_element
    return abbre_dict

In [7]:
output_f = open('/rdsgpfs/general/user/yh4218/home/outputs/abbre/list_of_abbre_from_author.txt','w',encoding='UTF-8')
for file in file_list:
    PMC_number=file.split('/')[11]
    #print ('——————————————————————'+PMC_number+'——————————————————————')
    output_f.write(PMC_number+'\n')
    with open(file,'r',encoding='UTF-8',errors='ignore') as f:
        text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
        abbre_dict=get_abbre_dict(soup,PMC_number)
        if abbre_dict:
            for SF,LF in abbre_dict.items():
                output_f.write(SF.strip()+'\t'+LF.strip().strip('.')+'\n')
                #print (SF+':'+LF)
output_f.close()

# Extract and Write main text

In [7]:
discard_list=['funding',
              'Conflicts of Interest',
              #'Additional Information',
              'Acknowledgment',
              'Acknowledgement',
              'financial',
              'contribution',
              'conflict',
              'footnotes',
              'interest',
              'Reference',
              'Formats',
              'Share',
              'Patent',
              'disclosure',
              'Availability',
              'additional']
abstract_list=['abstract']
introduction_list=['introduction']
background_list=['background']
overview_list=['overview']
supplementary_material_list=['supplementary','Associated Data','Supplemental']
abbreviation_list=['glossary','abbreviation']
method_material_list=['method','material','Experimental'] # sometimes titled Experimental section & procedure 
result_discussion_list=['Results','discussion']
conclusion_list=['conclusion']  # sometimes with future direction
keep_list=abstract_list+introduction_list+background_list+overview_list+supplementary_material_list+method_material_list+result_discussion_list+conclusion_list
# test extraction from abbre given by author
#keep_list=abbreviation_list

def extract_particular_paragraph(soup_og,filepath):
    main_text=''
    paragraph = soup_og.find_all('p',attrs={'id': re.compile('(__|_|)(p|P|Par|par|idm|para)\d+')})
    for number, element in enumerate(paragraph):
        nearest_tag = element.previous_element
        check = False
        while nearest_tag:
            if nearest_tag.name != 'h2':
                nearest_tag = nearest_tag.previous_element
            else:
                for keep_word in keep_list:
                    if re.search(keep_word,nearest_tag.get_text(),re.IGNORECASE):
                        for discard_word in discard_list:
                            if not re.search(discard_word,nearest_tag.get_text(),re.IGNORECASE):
                                check = True                                
                break
                
        if check:
            #element.attrs['style'] ="color:red;"
            main_text+=element.get_text().replace('\n',' ')

    return main_text

In [8]:
def mergeDict(dict1, dict2):
    dict3 = {**dict1, **dict2}
    for key, value in dict3.items():
        if key in dict1 and key in dict2:
            dict3[key] = [value , dict1[key]]
    return dict3

In [21]:
def merge_pair_Dict(pairs,whole_pair_dict):
    for SF, LF in pairs.items():
        if SF in list(whole_pair_dict.keys()):
            if LF.lower() not in whole_pair_dict[SF]:
                whole_pair_dict[SF].append(LF.lower())
        else:
            whole_pair_dict[SF]=[LF.lower()]
    return whole_pair_dict

# Extract abbre

In [24]:
# reload modified SH package
import importlib
importlib.reload(schwartz_hearst)

main_text_list=[]
abbre_extraction_dict={}
text_to_be_written=''
whole_dict={}
for file in file_list:
    with open(file,'r',encoding='UTF-8',errors='ignore') as f:
        text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
        main_text=extract_particular_paragraph(soup,file)
        with open(file.replace('.html','.txt'),'w') as f2:
            f2.write(main_text)
        main_text_list.append(main_text)
        # highlight in html
        with open(file.replace('.html','_highlighted.html'),'w') as f3:
            pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text=main_text,most_common_definition=True)
            soup_SH=highlight_SF_LF(pairs)
            f3.write(soup_SH)
            print (file.split('/')[-1]+'  :  '+str(pairs))
            whole_dict=merge_pair_Dict(pairs,whole_dict)
            

        text_to_be_written+=file.split('/')[-1]+'\n'
        for SF, LF in pairs.items():
            text_to_be_written += SF+'\t'+LF+'\n'
        
# write all result into one file
with open('/rdsgpfs/general/user/yh4218/home/outputs/abbre/list_of_abbre.txt','w') as f4:
    f4.write(text_to_be_written)

PMC3901241.html  :  {'H-MCR': 'Hierarchical multivariate curve resolution', 'OPLS-DA': 'orthogonal partial least squares discriminant analysis', 'GC/TOFMS': 'gas chromatography/time of flight mass spectrometry', 'MWAS': 'metabolome-wide associations', 'MCR': 'multivariate curve resolution', 'NMR': 'nuclear magnetic spectroscopy', 'BCAAs': 'branched chain amino acids', 'PCA': 'principal component analysis', 'OSC': 'orthogonal signal correction', 'CV': 'cross validation'}
PMC5372212.html  :  {'PCa': 'prostate cancer', 'PSA': 'prostate-specific antigen', 'DRE': 'digital rectal examination', 'PM': 'prostatic massage', 'GNMT': 'glycine N-methyl transferase', 'DMGDH': 'dimethylglycine dehydrogenase', 'SARDH': 'sarcosine dehydrogenase', 'PCA3': 'PCa antigen 3', 'TDO': 'tryptophan 2,3-dioxygenase', 'IDO': 'indoleamine 2,3-dioxygenase', 'KAT': 'kynurenine aminotransferase', 'ACN': 'Acetonitrile', 'RP': 'radical prostatectomy', 'MRM': 'Multiple Reaction Monitoring', 'RSD, %': 'relative standard 

In [25]:
whole_dict

{'H-MCR': ['hierarchical multivariate curve resolution'],
 'OPLS-DA': ['orthogonal partial least squares discriminant analysis',
  'orthogonal projections to latent structures-discriminant analysis',
  'orthogonal partial least square discriminant analysis',
  'orthogonal projection to latent structures discriminant analysis',
  'orthogonal projection to latent structure discriminant analysis',
  'orthogonal partial least square-discriminant analysis',
  'orthogonal partial least squares-discriminant analysis',
  'orthogonal partial least-squares-discriminant anlaysis',
  'orthogonal partial least-squares discriminant analysis',
  'opls-discriminant analysis'],
 'GC/TOFMS': ['gas chromatography/time of flight mass spectrometry'],
 'MWAS': ['metabolome-wide associations',
  'metabolome-wide association studies',
  'metabolomics-wide association studies'],
 'MCR': ['multivariate curve resolution'],
 'NMR': ['nuclear magnetic spectroscopy',
  'nuclear magnetic resonance',
  'nuclear magne

In [26]:
fuzz.ratio('nonesterified fatty acids','non-esterified fatty acids')

98

In [103]:
# reload modified SH package
import importlib
importlib.reload(schwartz_hearst)

file_list=['/rdsgpfs/general/user/yh4218/home/outputs/abbre/NLP/NLP-MWAS/cancer (Trait)/PMC4884347.html']
for file in file_list:
    with open(file,'r',encoding='UTF-8',errors='ignore') as f:
        text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
        main_text=extract_particular_paragraph(soup,file).replace('\n',' ')
        #print (main_text)
        pairs = yanhu.extract_abbreviation_definition_pairs(doc_text=main_text,most_common_definition=True)
        print (pairs)

ALDOC aldolase C, fructose-bisphosphate; BMI, body mass index; CER, continuous energy restriction; DAVID, Database for Annotation, Visualization and Integrated Discovery; FDR, false discovery rate; GCMS, gas chromatography mass spectrometry; HDL, high-density lipoprotein; HOMA, homeostatic model assessment; IER, intermittent energy restriction; LCMS, liquid chromatography mass spectrometry; LDL, low-density lipoprotein; MB-ML-PLS-DA, multi-block multi-level partial least squares for discriminant analysis; PAM, prediction analysis of microarrays; SAM, significance analysis of microarrays; SCD, stearoyl-CoA desaturase; TCA, tricarboxylic acid cycle; TP, time point
{}


# SH method

In [131]:
# reload modified SH package
import importlib
importlib.reload(schwartz_hearst)

for main_text in main_text_list:
#    #main_text=highlight_particular_paragraph(soup)
    pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text=main_text,all_definition=True)
    print (pairs)

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [84]:
# Test

In [ ]:
# reload modified SH package
import importlib
importlib.reload(schwartz_hearst)
# test
for doc_text in main_text_list:
    pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text=doc_text)
    #print (doc_text)
    print (pairs)

# Ab3P method

In [46]:
subprocess.call('cd /rdsgpfs/general/user/yh4218/home/Ab3P; /rdsgpfs/general/user/yh4218/home/Ab3P/identify_abbr /rdsgpfs/general/user/yh4218/home/data/main_text.txt > /rdsgpfs/general/user/yh4218/home/data/test.txt',shell=True)

0

# Ab3P results to dict

In [47]:
f = open("/rdsgpfs/general/user/yh4218/home/data/test.txt", "r")
keys=[]
values=[]
for line in f.readlines()[1:]:
    if float(line.split('|')[2].strip())>0.97:
        #print (float(line.split('|')[2].strip()))
        keys.append(line.split('|')[0].strip())
        values.append(line.split('|')[1].strip())
abbre_dict=dict(zip(keys,values))
f.close()
print (abbre_dict)

{'IDIBAPS': 'Institut d’Investigacions Biomediques August Pi Sunyer', 'NAFLD': 'nonalcoholic fatty liver disease', 'UPLC-MS': 'ultra-performance liquid chromatography coupled to mass spectrometry', 'BMI': 'body-mass index', 'TAG': 'triacylglycerides', 'TOF': 'UPLC-time-of-flight', 'AC': 'acyl carnitines', 'DAG': 'diacylglycerides', 'ChoE': 'cholesterol esters', 'SM': 'sphingomyelin', 'Cer': 'ceramides', 'IS': 'internal standard', 'RSD': 'relative standard deviation', 'ROC': 'Receiver operating characteristic', 'PUFA': 'polyunsaturated fatty acid', 'VLDL': 'very low density lipoprotein', 'AUC': 'accuracy'}


# Highlight Abbre

In [12]:
def highlight_SF_LF(pairs):
    soup_new=str(soup)
    for key,value in pairs.items():
        #print(key+':'+value)
        soup_new=soup_new.replace(str(key),'<span style="background:yellow;">'+str(key)+'</span>') # Color All SF
        #soup_new=soup_new.replace(str(value),'<span style="background:yellow;">'+str(value)+'</span>') # Color All LF
    return soup_new

# Write html

In [49]:
soup_SH=highlight_SF_LF(pairs)
soup_Ab3P=highlight_SF_LF(abbre_dict)

In [50]:
with open("output_SH.html", "w",encoding='UTF-8') as file:
    file.write(soup_SH)
with open("output_Ab3P.html", "w",encoding='UTF-8') as file:
    file.write(soup_Ab3P)

# backup

In [24]:
# reload modified SH package
import importlib
importlib.reload(schwartz_hearst)

main_text_list=[]
abbre_extraction_dict={}
text_to_be_written=''
for file in file_list:
    with open(file,'r',encoding='UTF-8',errors='ignore') as f:
        text = f.read()
        soup = BeautifulSoup(text, 'html.parser')
        main_text=extract_particular_paragraph(soup,file)
        with open(file.replace('.html','.txt'),'w') as f2:
            f2.write(main_text)
        main_text_list.append(main_text)
        # highlight in html
        with open(file.replace('.html','_highlighted.html'),'w') as f3:
            pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text=main_text,most_common_definition=True)
            soup_SH=highlight_SF_LF(pairs)
            f3.write(soup_SH)
            for SF, LF in pairs.items():
                if SF.islower():print (SF+':'+LF)
            #print (file.split('/')[-1]+'  :  '+str(pairs))

        text_to_be_written+=file.split('/')[-1]+'\n'
        for SF, LF in pairs.items():
            text_to_be_written += SF+'\t'+LF+'\n'
        
# write all result into one file
with open('/rdsgpfs/general/user/yh4218/home/outputs/abbre/list_of_abbre.txt','w') as f4:
    f4.write(text_to_be_written)

controls:collected before and after prostatic massage, and 15 healthy volunteers
range:ranges
rpm:revolutions per minute
red:representative drug-naïve patient with first-onset schizophrenia
red:represent dilution series of (blue) 540,
noesypr:NOESY-presaturation pulse sequence
microsomal:mitochondrial β-oxidation, peroxisomal β-oxidation, and endoplasmic reticulum
ii:in biological samples,
un-scaled:unit-variance or pareto scaling or the application of no secondary scaling
ppb:part per billion
ppt:parts per trillion
tm:time
metabotype:metabolic phenotype
b/i:blank with internal standard
percent:Patient characteristicsData are reported as mean ± standard deviation (SD) or number
p value:PV FunctionValues are β-coefficients
p values:PV FunctionValues are β-coefficients
cm:circumference
aldh1a1:Aldehyde dehydrogenase 1 family, member A1
glmnet:generalized linear model net
ncomp:number of components
mm:macro- and microscopically
hot spot:high-power (x40 objective) fields in the most prolif